# Исследование надежности заемщиков


## Загрузка данных и получение информации о данных.

In [77]:
import pandas as pd
from google.colab import data_table

data_table.enable_dataframe_formatter()

try:
  df = pd.read_csv('drive/MyDrive/datasets/data.csv')
except:
  df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

## Обзор полученных данных



In [78]:
display(df.head(10))
display(df.info())
df.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выводы по таблице.
Перед нами данные клиентов банка. В ней представлены такие данные как доход, семейное положение, образование и т. д. На основании этих данных попробуем провести анализ способности потенциального заемщика вернуть кредит банку. В представленных данных есть проблемы, которые необходимо решить: пропуски данных, странные данные в стаже клиента, разный регистр в образовании.

## Рассмотрим каждый столбец по отдельности. 


### Дубликаты
Для начала проверим и удалим дубликаты.

In [79]:
df = df.drop_duplicates().reset_index(drop=True)

df.duplicated().sum()

0

### Данные о детях

In [80]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей 20 и -1 выглядит странно. Скорей всего это опечатка и тут следует подставить 2 и 1 соответственно.

In [81]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Приведем данные об образовании к единому регистру.

In [82]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Семейное положение
Тут все в порядке.

In [83]:
df['family_status'].value_counts()


женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

### Статус клиента
Все ок.

In [84]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

### Пол

In [85]:
df['gender'].value_counts()
df.loc[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10690,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Странные данные в одном столбце. Удалим его.

In [86]:
df.drop(df.index[10701], inplace=True)

print(df['gender'].unique())
df.reset_index(drop=False)
print(df.index[10701])

['F' 'M' 'XNA']
10702


### Стаж клиента 
Стаж указан в днях. Некоторые данные с отрицательными значениями, их на до заменить на положительные. Некоторые значения некорректны. Например, стаж больше возраста. Для наглядности стаж лучше перевести в года.

In [87]:
df['days_employed'].describe()

count     19350.000000
mean      63049.879932
std      140830.165076
min      -18388.949901
25%       -2747.486276
50%       -1203.307346
75%        -291.077541
max      401755.400475
Name: days_employed, dtype: float64

Заменим отрицательные данные на положительные.

In [88]:
df['days_employed'] = df['days_employed'].abs()

display(df['days_employed'].head(10))

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

Данные свыше 20000 выглядят странно т.к. в годах это более 55 лет, что бывает крайне редко, а более 100 не встречается никогда. Скорей всего при выгрузке данных произошла ошибка и плавающая точка сместилась на две позиции. Так же переведем дни в года и отбросим дробную часть, таблицу переименуем. Попробуем восстановить данные.

In [89]:
df.loc[(df['days_employed'] > 20000), 'days_employed'] = df.loc[(df['days_employed'] > 25000), 'days_employed'] / 100


df['days_employed'] = df['days_employed'].fillna(0)
df['days_employed'] = df['days_employed'] / 365
df['days_employed'] = df['days_employed'].astype(int)
df.rename(columns = {'days_employed' : 'years_employed'}, inplace = True) 


print(df['years_employed'].describe())



count    21470.000000
mean         5.929949
std          5.931074
min          0.000000
25%          1.000000
50%          4.000000
75%          9.000000
max         50.000000
Name: years_employed, dtype: float64


### Доход клиента
Рассмотрим доход. Минимум 98000, максимум 226000 - это скорей всего рубли. Здесь так же имеются пропуски. Возможно часть из них выгрузилась неправильно, например у клиентов cо статусом "сотрудник" имеет смысл заполнить медианным значением. Остальным присвоим ноль, на анализ это не повлияет.


In [90]:
df.loc[(df['total_income'].isna())&(df['income_type'].str.lower() == 'сотрудник'),'total_income']\
  = df.loc[df['income_type'].str.lower() == 'сотрудник','total_income'].median()

df['total_income'] = df['total_income'].fillna(0)
df['total_income'] = df['total_income'].astype(int)

df.isnull().sum()

display(df.head(10))
df.describe()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


,children,years_employed,dob_years,education_id,family_status_id,debt,total_income
count,21470.000000,21470.000000,21470.000000,21470.000000,21470.000000,21470.000000,2.147000e+04
mean,0.480252,5.929949,43.279553,0.817187,0.973684,0.081090,1.580434e+05
std,0.755902,5.931074,12.574388,0.548520,1.421115,0.272979,1.042177e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,1.000000,33.000000,1.000000,0.000000,0.000000,9.885050e+04
50%,0.000000,4.000000,42.000000,1.000000,0.000000,0.000000,1.425870e+05
75%,1.000000,9.000000,53.000000,1.000000,1.000000,0.000000,1.957518e+05
max,5.000000,50.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Цели кредита


In [91]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Целей кредитов много, но из них можно выделить основные: недвижимость, автомобиль, образование, свадьба, ремонт. Произведем их лемматизацию.

In [92]:
credit_purpose_keys= []
credit_purpose_list = list(df.purpose.unique())
for value in range(len(credit_purpose_list)):
    if 'образован' in credit_purpose_list[value]:
        credit_purpose_keys.append('Образование')
    elif 'авто' in credit_purpose_list[value]:
        credit_purpose_keys.append('Автомобиль')
    elif 'свадь' in credit_purpose_list[value]:
        credit_purpose_keys.append('Свадьба')
    elif 'ремон' in credit_purpose_list[value]:
        credit_purpose_keys.append('Ремонт')
    else:
        credit_purpose_keys.append('Недвижимость')
        
check_data = set(credit_purpose_keys)
print(check_data)

{'Недвижимость', 'Свадьба', 'Ремонт', 'Автомобиль', 'Образование'}


In [93]:
credit_purpose_dict = dict(zip(credit_purpose_list, credit_purpose_keys))
df['purpose'] = df['purpose'].map(credit_purpose_dict)

df.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,Недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,Автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,Недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,Образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,Свадьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,Недвижимость
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,Недвижимость
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,Образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,Свадьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,Недвижимость


###  Ненужные столбцы
Такие сталбцы как education_id и family_status_id не несут ни какой информации или дублируют ее. Удалим их. Так же переименуем столбцы для наглядности.

In [94]:
df.drop(columns = ['education_id', 'family_status_id'], axis = 1, inplace=True)

display(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21470
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   children        21470 non-null  int64 
 1   years_employed  21470 non-null  int64 
 2   dob_years       21470 non-null  int64 
 3   education       21470 non-null  object
 4   family_status   21470 non-null  object
 5   gender          21470 non-null  object
 6   income_type     21470 non-null  object
 7   debt            21470 non-null  int64 
 8   total_income    21470 non-null  int64 
 9   purpose         21470 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


None

,children,years_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,женат / замужем,F,сотрудник,0,253875,Недвижимость
1,1,11,36,среднее,женат / замужем,F,сотрудник,0,112080,Автомобиль
2,0,15,33,среднее,женат / замужем,M,сотрудник,0,145885,Недвижимость
3,3,11,32,среднее,женат / замужем,M,сотрудник,0,267628,Образование
4,0,9,53,среднее,гражданский брак,F,пенсионер,0,158616,Свадьба
5,0,2,27,высшее,гражданский брак,M,компаньон,0,255763,Недвижимость
6,0,7,43,высшее,женат / замужем,F,компаньон,0,240525,Недвижимость
7,0,0,50,среднее,женат / замужем,M,сотрудник,0,135823,Образование
8,2,18,35,высшее,гражданский брак,F,сотрудник,0,95856,Свадьба
9,0,5,41,среднее,женат / замужем,M,сотрудник,0,144425,Недвижимость


In [95]:
df.rename(
    columns = {
    'children' : 'Дети', 
    'years_employed' : 'Стаж', 
    'dob_years' : 'Возраст', 
    'education' : 'Образование', 
    'family_status' : 'Семья', 
    'gender' : 'Пол', 
    'income_type' : 'Сфера деятельности', 
    'debt' : 'Просрочки', 
    'total_income' : 'Доход', 
    'purpose' : 'Цель кредита'
    }, 
    inplace = True
    )

df.head(10)

,Дети,Стаж,Возраст,Образование,Семья,Пол,Сфера деятельности,Просрочки,Доход,Цель кредита
0,1,23,42,высшее,женат / замужем,F,сотрудник,0,253875,Недвижимость
1,1,11,36,среднее,женат / замужем,F,сотрудник,0,112080,Автомобиль
2,0,15,33,среднее,женат / замужем,M,сотрудник,0,145885,Недвижимость
3,3,11,32,среднее,женат / замужем,M,сотрудник,0,267628,Образование
4,0,9,53,среднее,гражданский брак,F,пенсионер,0,158616,Свадьба
5,0,2,27,высшее,гражданский брак,M,компаньон,0,255763,Недвижимость
6,0,7,43,высшее,женат / замужем,F,компаньон,0,240525,Недвижимость
7,0,0,50,среднее,женат / замужем,M,сотрудник,0,135823,Образование
8,2,18,35,высшее,гражданский брак,F,сотрудник,0,95856,Свадьба
9,0,5,41,среднее,женат / замужем,M,сотрудник,0,144425,Недвижимость


Данные обработаны и приведены в надлежащий вид. Можно приступить к анализу.

# Анализ данных


## Зависимость между наличием детей и возвратом кредита в срок
Посчитаем процент клиентов, имеющих когда-либо просрочки по кредитам разделенные на группы в зависимости от количества детей.

In [96]:
depends_of_children = df.groupby('Дети')['Просрочки'].sum() / df.groupby('Дети')['Просрочки'].count() * 100

print(depends_of_children)

Дети
0    7.535800
1    9.163921
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: Просрочки, dtype: float64


Бездетные заемщики немного реже имеют задолженности по кредитам, но в целом все имеют примерно одинаковые показатели. Заемщиков с 5 детьми не считаем так как их крайне мало и невозможно объективно провести анализ.

## Зависимость между семейным положением и возвратом кредита в срок

In [97]:
depends_of_family = df.groupby('Семья')['Просрочки'].sum() / df.groupby('Семья')['Просрочки'].count() * 100

print(depends_of_family)

Семья
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.322441
женат / замужем          7.542126
Name: Просрочки, dtype: float64


Одинокие люди чаще всех задерживают выплаты, примерно в 10% случаях. Разведенные и люди в браке показываю примерно одинаковый результат невозврата на уровне 7.5%.

## Зависимость между уровнем дохода и возвратом кредита в срок

In [98]:
df['Доход'].quantile([0.5])

0.5    142587.0
Name: Доход, dtype: float64

Уровень дохода начинается с 86000 рублей и заканчивается 226000. Его можно условно разделить на 2 группы: "средний" до 150000 и высокий.

In [99]:
def income_status(monthly_income):
    if monthly_income <= 150000:
      return 'Средний уровень дохода'
    else:
      return 'Высокий уровень дохода'

df['Уровень_дохода'] = df['Доход'].apply(income_status)

df.head()

,Дети,Стаж,Возраст,Образование,Семья,Пол,Сфера деятельности,Просрочки,Доход,Цель кредита,Уровень_дохода
0,1,23,42,высшее,женат / замужем,F,сотрудник,0,253875,Недвижимость,Высокий уровень дохода
1,1,11,36,среднее,женат / замужем,F,сотрудник,0,112080,Автомобиль,Средний уровень дохода
2,0,15,33,среднее,женат / замужем,M,сотрудник,0,145885,Недвижимость,Средний уровень дохода
3,3,11,32,среднее,женат / замужем,M,сотрудник,0,267628,Образование,Высокий уровень дохода
4,0,9,53,среднее,гражданский брак,F,пенсионер,0,158616,Свадьба,Высокий уровень дохода


In [100]:
depends_of_income = df.groupby('Уровень_дохода')['Просрочки'].sum() / df.groupby('Уровень_дохода')['Просрочки'].count() * 100
print(depends_of_income)

Уровень_дохода
Высокий уровень дохода    7.905913
Средний уровень дохода    8.260763
Name: Просрочки, dtype: float64


Клиенты, имеющие более высокий доход отдают кредиты лучше, но разница незначительна.

## Зависимость целей кредита на его возврат в срок

In [101]:
depends_of_purpose = df.groupby('Цель кредита')['Просрочки'].sum() / df.groupby('Цель кредита')['Просрочки'].count() * 100

print(depends_of_purpose)

Цель кредита
Автомобиль      9.354689
Недвижимость    7.318507
Образование     9.217738
Ремонт          5.766063
Свадьба         7.969152
Name: Просрочки, dtype: float64


Клиенты, взявшие кредит на ремонт, платят лучше всего, их процент невозврата всего 5.7%. Самые необязательные являются клиенты, кто берет кредит на автомобиль. Их процент невозврата 9.3%. Возможно это связано с тем, что не редко автомобиль является импульсной покупкой. Кредиты на образование тоже имеют просрочку наравне с автокредитами, возможно это связано с отчислением из учебного заведения. Кредиты на недвижимость и свадьбу находятся на среднем уровне.